# PyTorch BYOS

## Pre-requisites

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments.  This can speed up iterative testing and debugging while using the same familiar Python SDK interface.  Just change your estimator's `train_instance_type` to `local` (or `local_gpu` if you're using an ml.p2 or ml.p3 notebook instance).

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU).

**Note, you can only run a single local notebook at one time.**

In [ ]:
!/bin/bash ./utils/setup.sh

## Overview

The **SageMaker Python SDK** helps you deploy your models for training and hosting in optimized, productions ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow, MXNet, PyTorch and Chainer. This tutorial focuses on how to create a convolutional neural network model to train the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) using **PyTorch in local mode**.

### Set up the environment

This notebook was created and tested on a single ml.p2.xlarge notebook instance.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).

In [1]:
import os
import sagemaker
print(sagemaker.__version__)

from sagemaker.pytorch import PyTorch
from sagemaker.pytorch.model import PyTorchModel

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-yolov7'

role = sagemaker.get_execution_role()

2.96.0


In [2]:
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = local


### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [8]:
base_dir = 'file:///home/ec2-user/SageMaker/yolov7_sagemaker/data/'
inputs = {'cfg': base_dir+'cfg/', 'weights': base_dir+'weights/', 'images': base_dir+'images/', 'labels': base_dir+'labels/'}
print(inputs)

{'cfg': 'file:///home/ec2-user/SageMaker/yolov7_sagemaker/data/cfg/', 'weights': 'file:///home/ec2-user/SageMaker/yolov7_sagemaker/data/weights/', 'images': 'file:///home/ec2-user/SageMaker/yolov7_sagemaker/data/images/', 'labels': 'file:///home/ec2-user/SageMaker/yolov7_sagemaker/data/labels/'}


## Script Functions

SageMaker invokes the main function defined within your training script for training. When deploying your trained model to an endpoint, the model_fn() is called to determine how to load your trained model. The model_fn() along with a few other functions list below are called to enable predictions on SageMaker.

### [Predicting Functions](https://github.com/aws/sagemaker-pytorch-containers/blob/master/src/sagemaker_pytorch_container/serving.py)
* model_fn(model_dir) - loads your model.
* input_fn(serialized_input_data, content_type) - deserializes predictions to predict_fn.
* output_fn(prediction_output, accept) - serializes predictions from predict_fn.
* predict_fn(input_data, model) - calls a model on data deserialized in input_fn.

The model_fn() is the only function that doesn't have a default implementation and is required by the user for using PyTorch on SageMaker. 

## Create a training job using the sagemaker.PyTorch estimator

The `PyTorch` class allows us to run our training function on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

After we've constructed our `PyTorch` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.


In [ ]:
git_config = {'repo': 'https://github.com/WongKinYiu/yolov7.git', 'branch': 'main'}

hyperparameters = {'data': '/opt/ml/input/data/cfg/coco128.yaml', 
                   'cfg': '/opt/ml/code/cfg/training/yolov7.yaml', 
#                    'hyp': '/opt/ml/code/data/hyp.scratch.p5.yaml', 
                   'weight': '/opt/ml/input/data/weights/yolov7.pt',
                   'project': '/opt/ml/model/',
                   'name': 'tutorial', 'img': 640, 'batch': 2, 'epochs': 5, 'workers': 1}  # set batch and workers to 1, becasue of shared memory issue in local mode
#                    'name': 'tutorial', 'img': 640, 'batch': 8, 'epochs': 5, 'workers': 1, 'device': '0,1,2,3,4,5,6,7'}  # set batch and workers to 1, becasue of shared memory issue in local mode

estimator = PyTorch(entry_point='train.py',
                            source_dir='.',
                            git_config=git_config,
                            role=role,
                            hyperparameters=hyperparameters,
                            framework_version='1.9.1',  # '1.8.1', '1.9.1'
                            py_version='py38',  # 'py3', 'py38'
                            script_mode=True,
                            instance_count=1,  # 1 or 2 or ...
                            instance_type=instance_type)

estimator.fit(inputs)

Creating nc0hdyraoc-algo-1-qdwti ... 
Creating nc0hdyraoc-algo-1-qdwti ... done
Attaching to nc0hdyraoc-algo-1-qdwti
nc0hdyraoc-algo-1-qdwti | 2022-08-02 10:12:36,839 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
nc0hdyraoc-algo-1-qdwti | 2022-08-02 10:12:36,840 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
nc0hdyraoc-algo-1-qdwti | 2022-08-02 10:12:36,848 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
nc0hdyraoc-algo-1-qdwti | 2022-08-02 10:12:36,851 sagemaker_pytorch_container.training INFO     Invoking user training script.
nc0hdyraoc-algo-1-qdwti | 2022-08-02 10:12:37,901 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
nc0hdyraoc-algo-1-qdwti | /opt/conda/bin/python3.8 -m pip install -r requirements.txt
nc0hdyraoc-algo-1-qdwti | Requirement already satisfied: matplotlib>=3.2.2 in /opt/conda/lib/python3.8/site-packages (from -

## SageMaker Training using GPU instance

In [15]:
WORK_DIRECTORY = '/home/ec2-user/SageMaker/yolov7_sagemaker/data/'
data_location = sagemaker_session.upload_data(WORK_DIRECTORY, key_prefix=prefix)
# data_location = 's3://{}/{}'.format(bucket, prefix)
inputs = {'cfg': data_location+'/cfg', 'weights': data_location+'/weights', 'images': data_location+'/images', 'labels': data_location+'/labels'}
print(inputs)

{'cfg': 's3://sagemaker-us-east-1-579019700964/sagemaker/DEMO-pytorch-yolov7/cfg', 'weights': 's3://sagemaker-us-east-1-579019700964/sagemaker/DEMO-pytorch-yolov7/weights', 'images': 's3://sagemaker-us-east-1-579019700964/sagemaker/DEMO-pytorch-yolov7/images', 'labels': 's3://sagemaker-us-east-1-579019700964/sagemaker/DEMO-pytorch-yolov7/labels'}


In [17]:
hyperparameters = {'data': '/opt/ml/input/data/cfg/coco128.yaml', 
                   'cfg': '/opt/ml/code/cfg/training/yolov7.yaml', 
#                    'hyp': '/opt/ml/code/data/hyp.scratch.p5.yaml', 
                   'weight': '/opt/ml/input/data/weights/yolov7.pt',
                   'project': '/opt/ml/model/',
                   'name': 'tutorial', 'img': 640, 'batch': 16, 'epochs': 5}  # Single CPU or GPU
#                    'name': 'tutorial', 'img': 640, 'batch': 16*8, 'epochs': 5, 'device': '0,1,2,3,4,5,6,7'}  # Multi-GPU: DP Mode

instance_type = 'ml.p3.2xlarge'  # 'ml.p3.2xlarge' or 'ml.p3.8xlarge' or ...

git_config = {'repo': 'https://github.com/WongKinYiu/yolov7.git', 'branch': 'main'}

estimator = PyTorch(entry_point='train.py',
                            source_dir='.',
                            git_config=git_config,
                            role=role,
                            hyperparameters=hyperparameters,
                            framework_version='1.9.1',  # '1.8.1', '1.9.1'
                            py_version='py38',  # 'py3', 'py38'
                            script_mode=True,
                            instance_count=1,  # 1 or 2 or ...
                            instance_type=instance_type)

estimator.fit(inputs)

2022-08-05 03:10:34 Starting - Starting the training job...
2022-08-05 03:11:00 Starting - Preparing the instances for trainingProfilerReport-1659669034: InProgress
.........
2022-08-05 03:12:22 Downloading - Downloading input data...
2022-08-05 03:12:58 Training - Downloading the training image.............................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-05 03:17:48,274 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-08-05 03:17:48,296 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-05 03:17:48,304 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-08-05 03:17:50,083 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python -m pip install -r requirements.txt
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 64.0 MB/s eta 0:00

In [18]:
training_job_name = estimator.latest_training_job.name
# training_job_name = 'pytorch-training-2022-03-03-04-38-25-840'
print(training_job_name)

pytorch-training-2022-08-05-03-10-28-450


# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint (in this case locally) which serves prediction requests in real-time.

In [19]:
!rm -rf model.tar.gz
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz .
!tar -xvf model.tar.gz

download: s3://sagemaker-us-east-1-579019700964/pytorch-training-2022-08-05-03-10-28-450/output/model.tar.gz to ./model.tar.gz
tutorial/
tutorial/R_curve.png
tutorial/train_batch7.jpg
tutorial/results.txt
tutorial/train_batch9.jpg
tutorial/train_batch3.jpg
tutorial/weights/
tutorial/weights/epoch_002.pt
tutorial/weights/epoch_001.pt
tutorial/weights/best.pt
tutorial/weights/epoch_004.pt
tutorial/weights/last.pt
tutorial/weights/init.pt
tutorial/weights/epoch_003.pt
tutorial/weights/epoch_000.pt
tutorial/train_batch5.jpg
tutorial/train_batch6.jpg
tutorial/test_batch2_pred.jpg
tutorial/test_batch1_pred.jpg
tutorial/events.out.tfevents.1659669487.algo-1.52.0
tutorial/hyp.yaml
tutorial/PR_curve.png
tutorial/train_batch2.jpg
tutorial/test_batch0_pred.jpg
tutorial/test_batch2_labels.jpg
tutorial/confusion_matrix.png
tutorial/test_batch1_labels.jpg
tutorial/train_batch8.jpg
tutorial/train_batch0.jpg
tutorial/F1_curve.png
tutorial/train_batch1.jpg
tutorial/opt.yaml
tutorial/P_curve.png
tutoria

In [24]:
!rm -rf model-pytorch.tar.gz
!cp tutorial/weights/best.pt model/
!cd model && tar -czvf ../model-pytorch.tar.gz *

!aws s3 cp model-pytorch.tar.gz s3://$bucket/$training_job_name/output/model-pytorch.tar.gz

best.pt
code/
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/inference-checkpoint.py
code/.ipynb_checkpoints/requirements-checkpoint.txt
code/inference.py
code/requirements.txt
upload: ./model-pytorch.tar.gz to s3://sagemaker-us-east-1-579019700964/pytorch-training-2022-08-05-03-10-28-450/output/model-pytorch.tar.gz


In [ ]:
# instance_type = 'local'
instance_type = 'ml.m5.xlarge'

# predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

pytorch_model = PyTorchModel(model_data='s3://{}/{}/output/model-pytorch.tar.gz'.format(bucket, training_job_name), role=role,
                             entry_point='inference.py', framework_version='1.9.1', py_version='py38')  # TODO set model_server_workers=1 to avoid torchhub bug, model_server_workers=1
predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

# Invoking the endpoint

In [27]:
import cv2
image = cv2.imread('data/images/inference/bus.jpg')

# print('image:', type(image), image.shape, image.dtype)
outputs = predictor.predict(image)
print('outputs: ', outputs)

KeyboardInterrupt: 

## Deploy the trained model to SageMaker Endpoint Serverless (Preview)

In [ ]:
!aws s3 cp model-pytorch.tar.gz s3://$bucket/$training_job_name/output/model.tar.gz

In [ ]:
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=1,
)

image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=sagemaker_session.boto_region_name,
    version="1.9.1",  # "1.8.1", "1.9.1"
    py_version="py38",  # "py3", "py38"
    instance_type="ml.m5.large",
    image_scope="inference",
)
print(image_uri)

estimator = PyTorch.attach(training_job_name=training_job_name)
serverless_predictor = estimator.deploy(serverless_inference_config=serverless_config, image_uri=image_uri, entry_point='inference.py')  # , model_server_workers=1

# pytorch_model = PyTorchModel(model_data='s3://{}/{}/output/model-pytorch.tar.gz'.format(bucket, training_job_name), role=role,
#                              entry_point='inference.py', framework_version='1.9.1', py_version='py38')  # TODO set model_server_workers=1 to avoid torchhub bug, model_server_workers=1
# serverless_predictor = pytorch_model.deploy(serverless_inference_config=serverless_config, image_uri=image_uri)

In [12]:
import cv2
image = cv2.imread('data/images/inference/bus.jpg')

# print('image:', type(image), image.shape, image.dtype)
outputs = serverless_predictor.predict(image)
print('outputs: ', outputs)

NameError: name 'serverless_predictor' is not defined

# Clean-up

Deleting the local endpoint when you're finished is important since you can only run one local endpoint at a time.

In [28]:
# estimator.delete_endpoint()
predictor.delete_endpoint()

In [ ]:
serverless_predictor.delete_endpoint()